In [1]:
import cv2
import numpy as np
import pickle
import os

In [2]:
mask_dir = "/home/pengkang/Downloads/GenDataset/data/mask/"
trimap_dir = "/home/pengkang/Downloads/GenDataset/data/trimap/"

In [3]:
def erode_dilate(msk, struc="ELLIPSE", size=(3, 3)):
    if struc == "RECT":
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, size)
    elif struc == "CORSS":
        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, size)
    else:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, size)


    n_fg = (msk!=255)
    msk[msk>0]=255
    msk = msk/255.

    dilated = cv2.dilate(msk, kernel, iterations=np.random.randint(1, 20)) * 255

    cnt1 = len(np.where(msk >= 0)[0])
    cnt2 = len(np.where(msk == 0)[0])
    cnt3 = len(np.where(msk == 1)[0])
    # print("all:{} bg:{} fg:{}".format(cnt1, cnt2, cnt3))
    assert(cnt1 == cnt2 + cnt3)

    cnt1 = len(np.where(dilated >= 0)[0])
    cnt2 = len(np.where(dilated == 0)[0])
    cnt3 = len(np.where(dilated == 255)[0])
    #print("all:{} bg:{} fg:{}".format(cnt1, cnt2, cnt3))
    assert(cnt1 == cnt2 + cnt3)

    res = dilated.copy()
    #res[((dilated == 255) & (msk == 0))] = 128
    res[(dilated == 255) &  n_fg]=128
    return res.astype(np.uint8)

In [4]:
with open("/home/pengkang/Downloads/GenDataset/data/new_image_list.pkl",'rb') as f:
    data = pickle.load(f)

In [5]:
len(data)

17

In [6]:
data[0]['name']

'0_0.png'

In [7]:
os.system('rm ' + trimap_dir + '*.png')

256

In [8]:
for i in range(len(data)):
        msk_name = mask_dir + data[i]['name']
        # print(msk_name)
        trimap_name = trimap_dir + data[i]['name']
        msk = cv2.imread(msk_name, 0)
        trimap = erode_dilate(msk)
        #print("Write to {}".format(trimap_name))
        cv2.imwrite(trimap_name, trimap)
        if (i+1) % 500 ==0:
            print (i)